# EXAFS_plotting

Standalone notebook for EXAFS summary plots in E-space, k-space, R-space, and Cauchy wavelet magnitude (2D and 3D).

In [1]:
# Step 1: load EXAFS data into `data`
from pathlib import Path
import numpy as np
from larch import Group

filename = 'Copper Foil_2001_06_26_Scan1_idho4ix8.dat'
candidates = [
    Path(filename),
    Path('materials/Larch_Cu_foil_first_shell_example') / filename,
    Path('/Users/juanjuanhuang/Desktop/Python/Dr.XAS/materials/Larch_Cu_foil_first_shell_example') / filename,
]

data_path = next((p for p in candidates if p.exists()), None)
if data_path is None:
    raise FileNotFoundError(f'Could not find {filename}. Checked: {candidates}')

arr = np.loadtxt(data_path, comments='#')
if arr.ndim == 1:
    arr = arr[np.newaxis, :]

# User requested: energy = first column, mu = last column
data = Group(energy=arr[:, 0], mu=arr[:, -1], filename=str(data_path), label=data_path.name)
print(f'Loaded data from: {data_path}')
print(f'Number of points: {len(data.energy)}')
print(f'Columns detected: {arr.shape[1]} (using first as energy, last as mu)')

Loaded data from: Copper Foil_2001_06_26_Scan1_idho4ix8.dat
Number of points: 408
Columns detected: 4 (using first as energy, last as mu)


In [2]:
# [Plotting 4] Summary EXAFS plots + Cauchy wavelet magnitude
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from larch.xafs import pre_edge, autobk, xftf, cauchy_wavelet

if 'data' not in globals():
    raise NameError("`data` is not defined. Run the data-loading cell first.")

# Ensure required EXAFS arrays exist on the group.
if not hasattr(data, 'norm'):
    pre_edge(data)
if not hasattr(data, 'chi'):
    autobk(data, rbkg=1.0, kweight=2)

# Build R-space arrays for a consistent summary view.
xftf(data, kmin=2, kmax=13, dk=1, kweight=2)

mu_e = data.mu
chi_k2 = data.chi * data.k**2
chir_mag = np.abs(data.chir_mag)

fig_summary = make_subplots(
    rows=1,
    cols=3,
    horizontal_spacing=0.11,
    subplot_titles=(
        'E-space: mu(E) vs Energy',
        'k-space: k^2 * chi(k)',
        'R-space: |chi(R)|',
    ),
)

fig_summary.add_trace(
    go.Scatter(x=data.energy, y=mu_e, mode='lines', name='mu(E)', line=dict(color='teal', width=2)),
    row=1, col=1,
)
fig_summary.add_trace(
    go.Scatter(x=data.k, y=chi_k2, mode='lines', name='k^2 chi(k)', line=dict(color='navy', width=2)),
    row=1, col=2,
)
fig_summary.add_trace(
    go.Scatter(x=data.r, y=chir_mag, mode='lines', name='|chi(R)|', line=dict(color='crimson', width=2)),
    row=1, col=3,
)

fig_summary.update_layout(
    title='EXAFS Summary in E, k, and R spaces',
    template='simple_white',
    height=430,
    width=1400,
    showlegend=False,
)
fig_summary.update_xaxes(title_text='Energy (eV)', row=1, col=1)
fig_summary.update_yaxes(title_text='mu(E)', row=1, col=1)
fig_summary.update_xaxes(title_text='k (A^-1)', row=1, col=2)
fig_summary.update_yaxes(title_text='k^2 chi(k)', row=1, col=2)
fig_summary.update_xaxes(title_text='R (A)', row=1, col=3)
fig_summary.update_yaxes(title_text='|chi(R)|', row=1, col=3)
fig_summary.show()

# Cauchy wavelet magnitude on the (R, k) grid.
cauchy_wavelet(data, kweight=2, rmax_out=10)
k_wave = data.k
r_wave = data.wcauchy_r
wmag = data.wcauchy_mag

fig_wave2d = go.Figure(
    data=[
        go.Heatmap(
            x=k_wave,
            y=r_wave,
            z=wmag,
            colorscale='Viridis',
            colorbar=dict(title='|W(k, R)|'),
        )
    ]
)
fig_wave2d.update_layout(
    title='2D Cauchy Wavelet Magnitude',
    template='simple_white',
    xaxis_title='k (A^-1)',
    yaxis_title='R (A)',
    width=950,
    height=520,
)
fig_wave2d.show()

fig_wave3d = go.Figure(
    data=[
        go.Surface(
            x=k_wave,
            y=r_wave,
            z=wmag,
            colorscale='Viridis',
            colorbar=dict(title='|W(k, R)|'),
        )
    ]
)
fig_wave3d.update_layout(
    title='3D Cauchy Wavelet Magnitude',
    template='simple_white',
    width=950,
    height=650,
    scene=dict(
        xaxis_title='k (A^-1)',
        yaxis_title='R (A)',
        zaxis_title='|W(k, R)|',
        camera=dict(eye=dict(x=1.7, y=1.6, z=0.8)),
    ),
)
fig_wave3d.show()

# Save plotted data to txt files
outdir = Path('EXAFS_plot_data_txt')
outdir.mkdir(parents=True, exist_ok=True)

np.savetxt(
    outdir / 'E_muE.txt',
    np.column_stack((data.energy, mu_e)),
    header='energy_eV muE',
)

np.savetxt(
    outdir / 'k_k2chik.txt',
    np.column_stack((data.k, chi_k2)),
    header='k_A^-1 k2chik',
)

np.savetxt(
    outdir / 'R_chiR.txt',
    np.column_stack((data.r, chir_mag)),
    header='R_A chiR_magnitude',
)

np.savetxt(
    outdir / 'wavelet_2D_matrix.txt',
    wmag,
    header='wavelet magnitude matrix; rows follow wavelet_R_axis.txt, columns follow wavelet_k_axis.txt',
)

# Axis files for the wavelet matrix
np.savetxt(outdir / 'wavelet_k_axis.txt', k_wave, header='k_A^-1')
np.savetxt(outdir / 'wavelet_R_axis.txt', r_wave, header='R_A')

print('Saved txt files to:', outdir.resolve())
print('- E_muE.txt')
print('- k_k2chik.txt')
print('- R_chiR.txt')
print('- wavelet_2D_matrix.txt')
print('- wavelet_k_axis.txt')
print('- wavelet_R_axis.txt')
